In [48]:
# import dtale
import os
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest

In [2]:
print(os.getcwd())
dpath = os.path.join(os.getcwd(), "..\\space_titanic_data")
print(dpath)

C:\ALEX\Kaggle\space_titanic
C:\ALEX\Kaggle\space_titanic\..\space_titanic_data


In [3]:
files1 = os.listdir(dpath)
files2 = [os.path.join(dpath, f) for f in files1]
files = [f for f in files2 if os.path.isfile(f)]
print(files)

['C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\sample_submission.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\test.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\space_titanic_data\\train.csv']


## Preprocess Functions

### Transfer all to ONE HOT ENCODER and Cabin break down to 3 types of data
### Missing cost values are set to 0

In [4]:
def get_train_data_transform_rev1():
    df_ = pd.read_csv(files[2])  # the training data
    dft_ = transform_data(df_)
    df_id = dft_.loc[:, dft_.columns == 'PassengerId']
    dfd_ = dft_.loc[:, dft_.columns != 'PassengerId']  # passanger id should not be input for training
    X = dfd_.loc[:, dfd_.columns != 'Transported']
    Y = dfd_[['Transported']]    
    return X, Y, df_id

def get_train_data_transform_rev2():
    df_ = pd.read_csv(files[2])  # the training data
    dft_ = transform_data_rev2(df_)
    df_id = dft_.loc[:, dft_.columns == 'PassengerId']
    dfd_ = dft_.loc[:, dft_.columns != 'PassengerId']  # passanger id should not be input for training
    X = dfd_.loc[:, dfd_.columns != 'Transported']
    Y = dfd_[['Transported']]    
    return X, Y, df_id

def get_test_data_transform_rev1():
    df_ = pd.read_csv(files[1])  # the test data
    dft_ = transform_data(df_)
    df_id = dft_.loc[:, dft_.columns == 'PassengerId']
    X = dft_.loc[:, dft_.columns != 'PassengerId']  # passanger id should not be input for training
    return X, df_id

def get_test_data_transform_rev2():
    df_ = pd.read_csv(files[1])  # the test data
    dft_ = transform_data_rev2(df_)
    df_id = dft_.loc[:, dft_.columns == 'PassengerId']
    X = dft_.loc[:, dft_.columns != 'PassengerId']  # passanger id should not be input for training
    return X, df_id

In [5]:
def transform_data(df_):
    df = df_.copy()
    
    df['Home_Earth'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Earth' else 0, axis=1)
    df['Home_Europa'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Europa' else 0, axis=1)
    df['Home_Mars'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Mars' else 0, axis=1)
    df['cryo_sleep_true'] = df.apply(lambda row: 1 if row['CryoSleep'] == True else 0, axis=1)
    df['cryo_sleep_false'] = df.apply(lambda row: 1 if row['CryoSleep'] == False else 0, axis=1)    
    df['destination_cancri'] = df.apply(lambda row: 1 if row['Destination'] == '55 Cancri e' else 0, axis=1)
    df['destination_pso'] = df.apply(lambda row: 1 if row['Destination'] == 'PSO J318.5-22' else 0, axis=1)
    df['destination_trappist'] = df.apply(lambda row: 1 if row['Destination'] == 'TRAPPIST-1e' else 0, axis=1)    
    df['vip_true'] = df.apply(lambda row: 1 if row['VIP'] == True else 0, axis=1)
    df['vip_false'] = df.apply(lambda row: 1 if row['VIP'] == False else 0, axis=1)    
    
    ccc = df['Cabin'].str.split("/").values
    c = [k[0] for k in ccc if type(k) == list]
    cc = list(set(c))
    cc.sort()
    cc_d = {cc[i]:(i+1) for i in range(len(cc))}
    s = [k[2] for k in ccc if type(k) == list]
    ss = list(set(s))
    ss.sort()
    ss_d = {ss[i]:(i+1) for i in range(len(ss))}
    df['cabin_row_'] = df.apply(lambda row: cc_d[row['Cabin'].split("/")[0]] if type(row['Cabin']) == str else 0, axis=1)
    m = max(df['cabin_row_'].values)
    df['cabin_row'] = df['cabin_row_'] / m
    df['cabin_t'] = df.apply(lambda row: 1 if type(row['Cabin']) == str and row['Cabin'].split("/")[0] == 'T' else 0, axis=1)
    df['cabin_num_'] = df.apply(lambda row: int(row['Cabin'].split("/")[1]) if type(row['Cabin']) == str else 0, axis=1)
    m = max(df['cabin_num_'].values)
    df['cabin_num'] = df['cabin_num_'] / m
    df['cabin_side_'] = df.apply(lambda row: ss_d[row['Cabin'].split("/")[2]] if type(row['Cabin']) == str else 0, axis=1)    
    m = max(df['cabin_side_'].values)
    df['cabin_side'] = df['cabin_side_'] / m
    df = df.drop(['cabin_row_', 'cabin_num_', 'cabin_side_'], axis=1)
    
    max_age = max(df['Age'].values)
    df['age_norm'] = df.apply(lambda row: row['Age']/max_age if not pd.isnull(row['Age']) else 0, axis=1)    
    dfc = df.drop(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP'], axis=1)
    
    dfc['RoomService'].fillna(0, inplace=True)
    dfc['roomservice'] = dfc['RoomService'] / max(dfc['RoomService'].values)
    dfc['FoodCourt'].fillna(0, inplace=True)
    dfc['foodcourt'] = dfc['FoodCourt'] / max(dfc['FoodCourt'].values)
    dfc['ShoppingMall'].fillna(0, inplace=True)
    dfc['shoppingmall'] = dfc['ShoppingMall'] / max(dfc['ShoppingMall'].values)
    dfc['Spa'].fillna(0, inplace=True)
    dfc['spa'] = dfc['Spa'] / max(dfc['Spa'].values)
    dfc['VRDeck'].fillna(0, inplace=True)
    dfc['vrdeck'] = dfc['VRDeck'] / max(dfc['VRDeck'].values)
    dfc = dfc.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'], axis=1)
    
    return dfc

### Same as above but missing costs are imputed using KNN instead of set to 0

In [6]:
def transform_data_rev2(df_):
    df = df_.copy()
    
    df['Home_Earth'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Earth' else 0, axis=1)
    df['Home_Europa'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Europa' else 0, axis=1)
    df['Home_Mars'] = df.apply(lambda row: 1 if row['HomePlanet'] == 'Mars' else 0, axis=1)
    df['cryo_sleep_true'] = df.apply(lambda row: 1 if row['CryoSleep'] == True else 0, axis=1)
    df['cryo_sleep_false'] = df.apply(lambda row: 1 if row['CryoSleep'] == False else 0, axis=1)    
    df['destination_cancri'] = df.apply(lambda row: 1 if row['Destination'] == '55 Cancri e' else 0, axis=1)
    df['destination_pso'] = df.apply(lambda row: 1 if row['Destination'] == 'PSO J318.5-22' else 0, axis=1)
    df['destination_trappist'] = df.apply(lambda row: 1 if row['Destination'] == 'TRAPPIST-1e' else 0, axis=1)    
    df['vip_true'] = df.apply(lambda row: 1 if row['VIP'] == True else 0, axis=1)
    df['vip_false'] = df.apply(lambda row: 1 if row['VIP'] == False else 0, axis=1)    
    
    ccc = df['Cabin'].str.split("/").values
    c = [k[0] for k in ccc if type(k) == list]
    cc = list(set(c))
    cc.sort()
    cc_d = {cc[i]:(i+1) for i in range(len(cc))}
    s = [k[2] for k in ccc if type(k) == list]
    ss = list(set(s))
    ss.sort()
    ss_d = {ss[i]:(i+1) for i in range(len(ss))}
    df['cabin_row_'] = df.apply(lambda row: cc_d[row['Cabin'].split("/")[0]] if type(row['Cabin']) == str else 0, axis=1)
    m = max(df['cabin_row_'].values)
    df['cabin_row'] = df['cabin_row_'] / m
    df['cabin_t'] = df.apply(lambda row: 1 if type(row['Cabin']) == str and row['Cabin'].split("/")[0] == 'T' else 0, axis=1)
    df['cabin_num_'] = df.apply(lambda row: int(row['Cabin'].split("/")[1]) if type(row['Cabin']) == str else 0, axis=1)
    m = max(df['cabin_num_'].values)
    df['cabin_num'] = df['cabin_num_'] / m
    df['cabin_side_'] = df.apply(lambda row: ss_d[row['Cabin'].split("/")[2]] if type(row['Cabin']) == str else 0, axis=1)    
    m = max(df['cabin_side_'].values)
    df['cabin_side'] = df['cabin_side_'] / m
    df = df.drop(['cabin_row_', 'cabin_num_', 'cabin_side_'], axis=1)
    
    max_age = max(df['Age'].values)
    df['age_norm'] = df.apply(lambda row: row['Age']/max_age if not pd.isnull(row['Age']) else 0, axis=1)    
    dfc = df.drop(['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP'], axis=1)
    
    X = dfc.loc[:, (dfc.columns != 'Transported') & (dfc.columns != 'PassengerId') & (dfc.columns != 'Name')]
    xcols = list(X.columns)
    if 'Transported' in list(dfc.columns):
        Y = dfc[['Transported', 'PassengerId', 'Name']]
    else:
        Y = dfc[['PassengerId', 'Name']]
        
    imputer = KNNImputer()
    imputer.fit(X)
    Xtrans = imputer.transform(X)
    xtrans_df = pd.DataFrame(data=Xtrans, columns=xcols)
    dfc = pd.concat([xtrans_df, Y], axis=1)
    dfc['RoomService'].fillna(0, inplace=True)
    dfc['roomservice'] = dfc['RoomService'] / max(dfc['RoomService'].values)
    dfc['FoodCourt'].fillna(0, inplace=True)
    dfc['foodcourt'] = dfc['FoodCourt'] / max(dfc['FoodCourt'].values)
    dfc['ShoppingMall'].fillna(0, inplace=True)
    dfc['shoppingmall'] = dfc['ShoppingMall'] / max(dfc['ShoppingMall'].values)
    dfc['Spa'].fillna(0, inplace=True)
    dfc['spa'] = dfc['Spa'] / max(dfc['Spa'].values)
    dfc['VRDeck'].fillna(0, inplace=True)
    dfc['vrdeck'] = dfc['VRDeck'] / max(dfc['VRDeck'].values)
    dfc = dfc.drop(['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name'], axis=1)
    
    return dfc

# The Models - exploring models

In [7]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [39]:
test_size = 0.25

### The Data columns split

In [9]:
X, Y, df_id = get_train_data_transform_rev1()

## Training

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
model = XGBClassifier(n_estimators=65, num_parallel_tree=40)  # , num_parallel_tree=10)  #, n_jobs=100)

In [31]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=70, n_jobs=None, num_parallel_tree=100,
              predictor=None, random_state=None, ...)

## Model Predictions

In [28]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

In [29]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 79.51%


In [45]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))    

Accuracy: 81.83%
Accuracy: 80.91%
Accuracy: 79.58%
Accuracy: 79.81%
Accuracy: 80.31%
Accuracy: 78.89%
Accuracy: 79.81%
Accuracy: 80.17%
Accuracy: 79.25%
Accuracy: 80.04%


## Version 2

In [46]:
X2, Y2, df_id = get_train_data_transform_rev2()

In [47]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=test_size)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))    

Accuracy: 79.99%
Accuracy: 80.31%
Accuracy: 79.35%
Accuracy: 81.05%
Accuracy: 78.56%
Accuracy: 79.53%
Accuracy: 81.92%
Accuracy: 79.25%
Accuracy: 79.71%
Accuracy: 79.58%


## Train-Validate with PCA

In [ ]:
pca = PCA(n_components =0.9)
Xt = pca.fit_transform(X)

In [ ]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=test_size)
    pca = PCA()
    Xt = pca.fit_transform(X_train)
    model.fit(Xt, y_train)
    Xt_test = pca.transform(X_test)
    y_pred = model.predict(Xt_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))    

## Rev3

In [ ]:
iso = IsolationForest(contamination=0.1)

In [ ]:
dfc2 = transform_data_rev2(df)
dfd2 = dfc2.loc[:, dfc2.columns != 'PassengerId']  # passanger id should not be input for training
X2 = dfd2.loc[:, dfd2.columns != 'Transported']
Y2 = dfd2[['Transported']]

In [ ]:
len(X2)

In [ ]:
for i in range(10):
    X_train_, X_test, y_train_, y_test = train_test_split(X2, Y2, test_size=test_size)
    
    yhat = iso.fit_predict(X_train_)
    mask = yhat != -1
    X_train, y_train = X_train_.loc[mask, :], y_train_.loc[mask, :]
#     print(f"pre rows: {y_train_.shape} ;  post rows: {len(X_train)}")
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    predictions = [round(value) for value in y_pred]
    accuracy = accuracy_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0))    

#### Full Train

In [ ]:
yhat = iso.fit_predict(X2)
mask = yhat != -1
X2_train, y2_train = X2.loc[mask, :], Y2.loc[mask, :]
model.fit(X2_train, y2_train)

In [90]:
X1, Y1, df_idt = get_train_data_transform_rev1()
X2, Y2, df_idt = get_train_data_transform_rev2()
X1_t, df_id = get_test_data_transform_rev1()
X2_t, df_id = get_test_data_transform_rev2()

In [93]:
model1 = XGBClassifier(n_estimators=65, num_parallel_tree=40)  # , num_parallel_tree=10)  #, n_jobs=100)
model2 = XGBClassifier(n_estimators=65, num_parallel_tree=40)  # , num_parallel_tree=10)  #, n_jobs=100)

In [94]:
model1.fit(X1, Y1)
model2.fit(X2, Y2)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=65, n_jobs=None, num_parallel_tree=40,
              predictor=None, random_state=None, ...)

In [100]:
y1_pred1 = model1.predict(X1_t)
y1_pred2 = model1.predict(X2_t)
y2_pred1 = model2.predict(X1_t)
y2_pred2 = model2.predict(X2_t)

In [101]:
pred1_df1 = pd.DataFrame(data=y1_pred1, columns=['Transported'])
pred1_df1['Transported'] = pred1_df1['Transported'].astype(bool)

pred1_df2 = pd.DataFrame(data=y1_pred2, columns=['Transported'])
pred1_df2['Transported'] = pred1_df2['Transported'].astype(bool)

pred2_df1 = pd.DataFrame(data=y2_pred1, columns=['Transported'])
pred2_df1['Transported'] = pred2_df1['Transported'].astype(bool)

pred2_df2 = pd.DataFrame(data=y2_pred2, columns=['Transported'])
pred2_df2['Transported'] = pred2_df2['Transported'].astype(bool)

In [102]:
df1_test1 = pd.concat([df_id, pred1_df1], axis=1)
df1_test2 = pd.concat([df_id, pred1_df2], axis=1)
df2_test1 = pd.concat([df_id, pred2_df1], axis=1)
df2_test2 = pd.concat([df_id, pred2_df2], axis=1)

In [103]:
df1_test1.to_csv(f"xgboost11.csv", index=False)
df1_test2.to_csv(f"xgboost12.csv", index=False)
df2_test1.to_csv(f"xgboost21.csv", index=False)
df2_test2.to_csv(f"xgboost22.csv", index=False)

#  Models Predict
## Ensemble

In [49]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [50]:
bagging_kn = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
bagging_kn = BaggingClassifier(DecisionTreeClassifier(), max_samples=0.5, max_features=0.5)
bagging_ab = BaggingClassifier(AdaBoostClassifier(), max_samples=0.5, max_features=0.5)
bagging_gb = BaggingClassifier(GradientBoostingClassifier(), max_samples=0.5, max_features=0.5)
bagging_rf = BaggingClassifier(RandomForestClassifier(), max_samples=0.5, max_features=0.5)

In [61]:
estimators1 = [('K-Neighbors', bagging_kn), ('Decision-Trees', bagging_kn), ('AdaBoost', bagging_ab), 
               ('Grad-Boost', bagging_gb), ('Random-Forest', bagging_rf)]
estimators2 = [('K-Neighbors', bagging_kn), ('Decision-Trees', bagging_kn), ('AdaBoost', bagging_ab), 
               ('Grad-Boost', bagging_gb), ('Random-Forest', bagging_rf)]

In [52]:
import warnings
warnings.filterwarnings('ignore')

In [64]:
X1, Y1, df_id = get_train_data_transform_rev1()
X2, Y2, df_id = get_train_data_transform_rev2()

In [65]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=test_size)
#     yhat = iso.fit_predict(X_train_)
#     mask = yhat != -1
#     X_train, y_train = X_train_.loc[mask, :], y_train_.loc[mask, :]
    nn = []
    for (name, estimator) in estimators1:
        estimator.fit(X_train.values, y_train.values)
        y_pred = estimator.predict(X_test.values)
        accuracy = accuracy_score(y_test.values.reshape(len(y_test.values),), y_pred.reshape(len(y_pred),))
        nn.append([name, round(accuracy * 100.0, 2)])
    ss = [n[0]+": " + str(n[1]) + "%" for n in nn]
    s = "; ".join(ss)
    print(s)

K-Neighbors: 77.23%; Decision-Trees: 76.4%; AdaBoost: 78.56%; Grad-Boost: 78.84%; Random-Forest: 77.6%
K-Neighbors: 76.63%; Decision-Trees: 75.07%; AdaBoost: 78.24%; Grad-Boost: 79.21%; Random-Forest: 78.38%
K-Neighbors: 76.36%; Decision-Trees: 77.09%; AdaBoost: 78.43%; Grad-Boost: 78.98%; Random-Forest: 78.38%
K-Neighbors: 77.74%; Decision-Trees: 76.59%; AdaBoost: 79.67%; Grad-Boost: 79.3%; Random-Forest: 78.24%
K-Neighbors: 76.54%; Decision-Trees: 76.63%; AdaBoost: 77.23%; Grad-Boost: 78.24%; Random-Forest: 77.97%
K-Neighbors: 77.55%; Decision-Trees: 78.89%; AdaBoost: 79.53%; Grad-Boost: 80.4%; Random-Forest: 79.12%
K-Neighbors: 76.17%; Decision-Trees: 76.26%; AdaBoost: 78.24%; Grad-Boost: 78.38%; Random-Forest: 78.43%
K-Neighbors: 77.69%; Decision-Trees: 76.77%; AdaBoost: 78.15%; Grad-Boost: 80.68%; Random-Forest: 79.67%
K-Neighbors: 74.75%; Decision-Trees: 75.9%; AdaBoost: 78.84%; Grad-Boost: 79.99%; Random-Forest: 79.25%
K-Neighbors: 78.43%; Decision-Trees: 78.33%; AdaBoost: 80.5%

In [66]:
for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=test_size)
    
#     yhat = iso.fit_predict(X_train_)
#     mask = yhat != -1
#     X_train, y_train = X_train_.loc[mask, :], y_train_.loc[mask, :]
    nn = []
    for (name, estimator) in estimators2:
        estimator.fit(X_train.values, y_train.values)
        y_pred = estimator.predict(X_test.values)
        accuracy = accuracy_score(y_test.values.reshape(len(y_test.values),), y_pred.reshape(len(y_pred),))
        nn.append([name, round(accuracy * 100.0, 2)])
    ss = [n[0]+": " + str(n[1]) + "%" for n in nn]
    s = "; ".join(ss)
    print(s)

K-Neighbors: 77.69%; Decision-Trees: 76.77%; AdaBoost: 78.61%; Grad-Boost: 79.62%; Random-Forest: 79.85%
K-Neighbors: 76.82%; Decision-Trees: 75.94%; AdaBoost: 77.28%; Grad-Boost: 79.07%; Random-Forest: 77.51%
K-Neighbors: 77.92%; Decision-Trees: 75.67%; AdaBoost: 78.93%; Grad-Boost: 80.4%; Random-Forest: 79.53%
K-Neighbors: 78.52%; Decision-Trees: 76.08%; AdaBoost: 79.58%; Grad-Boost: 79.25%; Random-Forest: 79.39%
K-Neighbors: 78.47%; Decision-Trees: 77.64%; AdaBoost: 79.53%; Grad-Boost: 79.12%; Random-Forest: 79.16%
K-Neighbors: 77.87%; Decision-Trees: 76.86%; AdaBoost: 79.58%; Grad-Boost: 79.9%; Random-Forest: 78.24%
K-Neighbors: 77.05%; Decision-Trees: 77.78%; AdaBoost: 78.01%; Grad-Boost: 78.61%; Random-Forest: 77.05%
K-Neighbors: 77.51%; Decision-Trees: 76.91%; AdaBoost: 78.61%; Grad-Boost: 78.75%; Random-Forest: 78.66%
K-Neighbors: 77.41%; Decision-Trees: 76.36%; AdaBoost: 78.38%; Grad-Boost: 77.0%; Random-Forest: 77.09%
K-Neighbors: 77.74%; Decision-Trees: 76.91%; AdaBoost: 77.

In [67]:
X1_test, df_id = get_test_data_transform_rev1()
X2_test, df_id = get_test_data_transform_rev2()

In [ ]:
dft = pd.read_csv(files[1])  # the test data
dfct = transform_data(dft)
pass_id = dfct[['PassengerId']]
dfdt = dfct.loc[:, dfct.columns != 'PassengerId']  # passanger id should not be input for training
X_test = dfdt
print(len(X_test))

In [69]:
for (name, estimator) in estimators1:
    print(f"Start fitting {name}.")
    estimator.fit(X1.values, Y1.values)
    print(f"Done fitting {name}, now starting prediction of test.")
    
    y_pred = estimator.predict(X1_test.values)
    print(f"Done test prediction of {name}, now processing.")
    predictions = y_pred  # [round(value[0]) for value in y_pred]
    pred_df = pd.DataFrame(data=predictions, columns=['Transported'])
    pred_df['Transported'] = pred_df['Transported'].astype(bool)
    df_test = pd.concat([df_id, pred_df], axis=1)
    df_test.to_csv(f"submit_test revb1_test1 {name}.csv", index=False)

    y_pred = estimator.predict(X2_test.values)
    print(f"Done test prediction of {name}, now processing.")
    predictions = y_pred  # [round(value[0]) for value in y_pred]
    pred_df = pd.DataFrame(data=predictions, columns=['Transported'])
    pred_df['Transported'] = pred_df['Transported'].astype(bool)
    df_test = pd.concat([df_id, pred_df], axis=1)
    df_test.to_csv(f"submit_test revb1_test2 {name}.csv", index=False)    

Start fitting K-Neighbors.
Done fitting K-Neighbors, now starting prediction of test.
Done test prediction of K-Neighbors, now processing.
Done test prediction of K-Neighbors, now processing.
Start fitting Decision-Trees.
Done fitting Decision-Trees, now starting prediction of test.
Done test prediction of Decision-Trees, now processing.
Done test prediction of Decision-Trees, now processing.
Start fitting AdaBoost.
Done fitting AdaBoost, now starting prediction of test.
Done test prediction of AdaBoost, now processing.
Done test prediction of AdaBoost, now processing.
Start fitting Grad-Boost.
Done fitting Grad-Boost, now starting prediction of test.
Done test prediction of Grad-Boost, now processing.
Done test prediction of Grad-Boost, now processing.
Start fitting Random-Forest.
Done fitting Random-Forest, now starting prediction of test.
Done test prediction of Random-Forest, now processing.
Done test prediction of Random-Forest, now processing.


In [71]:
for (name, estimator) in estimators2:
    print(f"Start fitting {name}.")
    estimator.fit(X2.values, Y2.values)
    print(f"Done fitting {name}, now starting prediction of test.")
    
    y_pred = estimator.predict(X1_test.values)
    print(f"Done test prediction of {name}, now processing.")
    predictions = y_pred  # [round(value[0]) for value in y_pred]
    pred_df = pd.DataFrame(data=predictions, columns=['Transported'])
    pred_df['Transported'] = pred_df['Transported'].astype(bool)
    df_test = pd.concat([df_id, pred_df], axis=1)
    df_test.to_csv(f"submit_test revb2_test1 {name}.csv", index=False)

    y_pred = estimator.predict(X2_test.values)
    print(f"Done test prediction of {name}, now processing.")
    predictions = y_pred  # [round(value[0]) for value in y_pred]
    pred_df = pd.DataFrame(data=predictions, columns=['Transported'])
    pred_df['Transported'] = pred_df['Transported'].astype(bool)
    df_test = pd.concat([df_id, pred_df], axis=1)
    df_test.to_csv(f"submit_test revb2_test2 {name}.csv", index=False)    

Start fitting K-Neighbors.
Done fitting K-Neighbors, now starting prediction of test.
Done test prediction of K-Neighbors, now processing.
Done test prediction of K-Neighbors, now processing.
Start fitting Decision-Trees.
Done fitting Decision-Trees, now starting prediction of test.
Done test prediction of Decision-Trees, now processing.
Done test prediction of Decision-Trees, now processing.
Start fitting AdaBoost.
Done fitting AdaBoost, now starting prediction of test.
Done test prediction of AdaBoost, now processing.
Done test prediction of AdaBoost, now processing.
Start fitting Grad-Boost.
Done fitting Grad-Boost, now starting prediction of test.
Done test prediction of Grad-Boost, now processing.
Done test prediction of Grad-Boost, now processing.
Start fitting Random-Forest.
Done fitting Random-Forest, now starting prediction of test.
Done test prediction of Random-Forest, now processing.
Done test prediction of Random-Forest, now processing.


In [ ]:
# dft = pd.read_csv(files[1])  # the test data
# dfct = transform_data(dft)
# dfdt = dfct.loc[:, dfct.columns != 'PassengerId']  # passanger id should not be input for training
# X2 = dfdt
# print(len(X2))

# y_pred = model2.predict(X2)
# predictions = [round(value[0]) for value in y_pred]
# len(predictions)

# pass_id = dfct[['PassengerId']]
# pred_df = pd.DataFrame(data=predictions, columns=['Transported'])
# pred_df['Transported'] = pred_df['Transported'].astype(bool)
# df_test = pd.concat([pass_id, pred_df], axis=1)

# df_test.to_csv('submit_test rev6 nn full train.csv', index=False)

## NN Model

In [72]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras import Input

In [73]:
# x_in = Input(shape=(20,))
# x1 = Dense(64)(x_in)
# x2 = Dense(32)(x1)
# x3 = Dense(16)(x2)
# x_out = Dense(1)(x3)
# model = Model(inputs=x_in, outputs=x_out)

model = tf.keras.Sequential()
model.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(20,)))
model.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [76]:
nn_model1 = tf.keras.Sequential()
nn_model1.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(20,)))
nn_model1.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
nn_model1.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
nn_model1.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
nn_model1.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [77]:
nn_model2 = tf.keras.Sequential()
nn_model2.add(Dense(64, activation='relu', kernel_initializer='he_normal', input_shape=(20,)))
nn_model2.add(Dense(32, activation='relu', kernel_initializer='he_normal'))
nn_model2.add(Dense(16, activation='relu', kernel_initializer='he_normal'))
nn_model2.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
nn_model2.add(Dense(1, activation='sigmoid'))
nn_model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4), loss='binary_crossentropy', metrics=['accuracy'])

In [74]:
X1, Y1, df_id = get_train_data_transform_rev1()
X2, Y2, df_id = get_train_data_transform_rev2()

In [78]:
nn_model1.fit(X1, Y1, epochs=400, batch_size=32, verbose=1)

Epoch 1/400
272/272 [==============================] - 1s 1ms/step - loss: 0.6496 - accuracy: 0.5896
Epoch 2/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5640 - accuracy: 0.7137
Epoch 3/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5400 - accuracy: 0.7294
Epoch 4/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5298 - accuracy: 0.7393
Epoch 5/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5199 - accuracy: 0.7438
Epoch 6/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5130 - accuracy: 0.7461
Epoch 7/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5059 - accuracy: 0.7518
Epoch 8/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4989 - accuracy: 0.7560
Epoch 9/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4917 - accuracy: 0.7602
Epoch 10/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4855 - accura

272/272 [==============================] - 0s 1ms/step - loss: 0.3938 - accuracy: 0.8109
Epoch 82/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3932 - accuracy: 0.8133
Epoch 83/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3927 - accuracy: 0.8098
Epoch 84/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3910 - accuracy: 0.8110
Epoch 85/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3914 - accuracy: 0.8130
Epoch 86/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3899 - accuracy: 0.8142
Epoch 87/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3914 - accuracy: 0.8127
Epoch 88/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3909 - accuracy: 0.8163
Epoch 89/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3911 - accuracy: 0.8111
Epoch 90/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3903 - accuracy: 

272/272 [==============================] - 0s 1ms/step - loss: 0.3741 - accuracy: 0.8161
Epoch 161/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3748 - accuracy: 0.8165
Epoch 162/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3750 - accuracy: 0.8166
Epoch 163/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3734 - accuracy: 0.8188
Epoch 164/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3741 - accuracy: 0.8182
Epoch 165/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3725 - accuracy: 0.8200
Epoch 166/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3731 - accuracy: 0.8187
Epoch 167/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3745 - accuracy: 0.8172
Epoch 168/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3743 - accuracy: 0.8162
Epoch 169/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3721 - a

272/272 [==============================] - 0s 1ms/step - loss: 0.3586 - accuracy: 0.8233
Epoch 240/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3588 - accuracy: 0.8224
Epoch 241/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3587 - accuracy: 0.8248
Epoch 242/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3579 - accuracy: 0.8241
Epoch 243/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3586 - accuracy: 0.8247
Epoch 244/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3569 - accuracy: 0.8248
Epoch 245/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3578 - accuracy: 0.8250
Epoch 246/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3565 - accuracy: 0.8270
Epoch 247/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3577 - accuracy: 0.8253
Epoch 248/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3571 - a

272/272 [==============================] - 0s 1ms/step - loss: 0.3450 - accuracy: 0.8324
Epoch 319/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3466 - accuracy: 0.8294
Epoch 320/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3462 - accuracy: 0.8294
Epoch 321/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3454 - accuracy: 0.8288
Epoch 322/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3456 - accuracy: 0.8281
Epoch 323/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3456 - accuracy: 0.8287
Epoch 324/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3468 - accuracy: 0.8300
Epoch 325/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3449 - accuracy: 0.8299
Epoch 326/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3448 - accuracy: 0.8329
Epoch 327/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3456 - a

272/272 [==============================] - 0s 1ms/step - loss: 0.3368 - accuracy: 0.8381
Epoch 398/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3372 - accuracy: 0.8345
Epoch 399/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3358 - accuracy: 0.8373
Epoch 400/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3351 - accuracy: 0.8334


In [79]:
nn_model2.fit(X2, Y2, epochs=400, batch_size=32, verbose=1)

Epoch 1/400
272/272 [==============================] - 1s 1ms/step - loss: 0.6446 - accuracy: 0.6485
Epoch 2/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5494 - accuracy: 0.7287
Epoch 3/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5313 - accuracy: 0.7396
Epoch 4/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5217 - accuracy: 0.7424
Epoch 5/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5142 - accuracy: 0.7469
Epoch 6/400
272/272 [==============================] - 0s 1ms/step - loss: 0.5050 - accuracy: 0.7501
Epoch 7/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4986 - accuracy: 0.7551
Epoch 8/400
272/272 [==============================] - 0s 2ms/step - loss: 0.4917 - accuracy: 0.7588
Epoch 9/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4836 - accuracy: 0.7620
Epoch 10/400
272/272 [==============================] - 0s 1ms/step - loss: 0.4773 - accura

272/272 [==============================] - 0s 2ms/step - loss: 0.3885 - accuracy: 0.8102
Epoch 82/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3903 - accuracy: 0.8107
Epoch 83/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3876 - accuracy: 0.8109
Epoch 84/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3869 - accuracy: 0.8118
Epoch 85/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3881 - accuracy: 0.8100
Epoch 86/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3861 - accuracy: 0.8103
Epoch 87/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3869 - accuracy: 0.8104
Epoch 88/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3865 - accuracy: 0.8130
Epoch 89/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3871 - accuracy: 0.8113
Epoch 90/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3859 - accuracy: 

272/272 [==============================] - 0s 1ms/step - loss: 0.3653 - accuracy: 0.8208
Epoch 161/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3658 - accuracy: 0.8212
Epoch 162/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3655 - accuracy: 0.8247
Epoch 163/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3654 - accuracy: 0.8208
Epoch 164/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3643 - accuracy: 0.8220
Epoch 165/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3630 - accuracy: 0.8224
Epoch 166/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3650 - accuracy: 0.8230
Epoch 167/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3631 - accuracy: 0.8231
Epoch 168/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3648 - accuracy: 0.8223
Epoch 169/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3649 - a

272/272 [==============================] - 0s 1ms/step - loss: 0.3503 - accuracy: 0.8302
Epoch 240/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3512 - accuracy: 0.8292
Epoch 241/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3495 - accuracy: 0.8307
Epoch 242/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3504 - accuracy: 0.8296
Epoch 243/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3493 - accuracy: 0.8324
Epoch 244/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3495 - accuracy: 0.8285
Epoch 245/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3491 - accuracy: 0.8306
Epoch 246/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3487 - accuracy: 0.8319
Epoch 247/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3492 - accuracy: 0.8316
Epoch 248/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3475 - a

272/272 [==============================] - 0s 1ms/step - loss: 0.3397 - accuracy: 0.8372
Epoch 319/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8381
Epoch 320/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3374 - accuracy: 0.8377
Epoch 321/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3378 - accuracy: 0.8384
Epoch 322/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3398 - accuracy: 0.8335
Epoch 323/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3387 - accuracy: 0.8324
Epoch 324/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3370 - accuracy: 0.8350
Epoch 325/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8383
Epoch 326/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3388 - accuracy: 0.8338
Epoch 327/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3364 - a

272/272 [==============================] - 0s 2ms/step - loss: 0.3289 - accuracy: 0.8403
Epoch 398/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3283 - accuracy: 0.8448
Epoch 399/400
272/272 [==============================] - 0s 1ms/step - loss: 0.3266 - accuracy: 0.8415
Epoch 400/400
272/272 [==============================] - 0s 2ms/step - loss: 0.3290 - accuracy: 0.8406


In [80]:
X1_t, df_id = get_test_data_transform_rev1()
X2_t, df_id = get_test_data_transform_rev2()

In [81]:
y1_pred1 = nn_model1.predict(X1_t)
y1_pred2 = nn_model1.predict(X2_t)
y2_pred1 = nn_model2.predict(X1_t)
y2_pred2 = nn_model2.predict(X2_t)

134/134 [==============================] - 0s 729us/step


In [83]:
predictions11 = [round(value[0]) for value in y1_pred1]
predictions12 = [round(value[0]) for value in y1_pred2]
predictions21 = [round(value[0]) for value in y2_pred1]
predictions22 = [round(value[0]) for value in y2_pred2]

In [84]:
pred1_df1 = pd.DataFrame(data=predictions11, columns=['Transported'])
pred1_df2 = pd.DataFrame(data=predictions12, columns=['Transported'])
pred2_df1 = pd.DataFrame(data=predictions21, columns=['Transported'])
pred2_df2 = pd.DataFrame(data=predictions22, columns=['Transported'])

In [85]:
pred1_df1['Transported'] = pred1_df1['Transported'].astype(bool)
pred1_df2['Transported'] = pred1_df2['Transported'].astype(bool)
pred2_df1['Transported'] = pred2_df1['Transported'].astype(bool)
pred2_df2['Transported'] = pred2_df2['Transported'].astype(bool)

In [86]:
df1_test1 = pd.concat([df_id, pred1_df1], axis=1)
df1_test2 = pd.concat([df_id, pred1_df2], axis=1)
df2_test1 = pd.concat([df_id, pred2_df1], axis=1)
df2_test2 = pd.concat([df_id, pred2_df2], axis=1)

In [87]:
df1_test1.to_csv('submit nn11.csv', index=False)
df1_test2.to_csv('submit nn12.csv', index=False)
df2_test1.to_csv('submit nn21.csv', index=False)
df2_test2.to_csv('submit nn22.csv', index=False)

## Ensemble results from different models

In [104]:
en_path = '..\\newrev'

In [106]:
dpath = os.path.join(os.getcwd(), en_path)
files1 = os.listdir(dpath)
files2 = [os.path.join(dpath, f) for f in files1]
files_e = [f for f in files2 if os.path.isfile(f)]
print(files_e)

['C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\AdaBoost1.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\AdaBoost11.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\AdaBoost12.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\AdaBoost21.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\AdaBoost22.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Decision-Trees1.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Decision-Trees11.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Decision-Trees12.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Decision-Trees21.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Decision-Trees22.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Grad-Boost1.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Grad-Boost11.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Grad-Boost12.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Grad-Boost21.csv', 'C:\\ALEX\\Kaggle\\space_titanic\\..\\newrev\\Grad-Boost22.csv', 'C:\\ALEX\\Kaggle

In [107]:
fnam = [fn.rsplit("\\")[-1].split(".")[0] for fn in files_e]

In [109]:
dfe = None
for (file,m_name) in zip(files_e,fnam):
    df = pd.read_csv(file)
    if dfe is None:
        dfe = df
        dfe = dfe.rename(columns={"Transported": m_name})
    else:
        dfe[m_name] = df['Transported'].values

In [110]:
dfe

,PassengerId,AdaBoost1,AdaBoost11,AdaBoost12,AdaBoost21,AdaBoost22,Decision-Trees1,Decision-Trees11,Decision-Trees12,Decision-Trees21,...,Random-Forest21,Random-Forest22,xgboost11-0,xgboost11,xgboost12-0,xgboost12,xgboost21-0,xgboost21,xgboost22-0,xgboost22
0,0013_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False
1,0018_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0019_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,0021_01,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True
4,0023_01,True,False,False,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,True,True
4273,9269_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4274,9271_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4275,9273_01,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,True,True


In [111]:
dfe_sum = dfe.sum(axis=1)
dfe_cnt = dfe.count(axis=1)

In [112]:
dfe['sum'] = dfe_sum
dfe['count'] = dfe_cnt

In [113]:
dfe

,PassengerId,AdaBoost1,AdaBoost11,AdaBoost12,AdaBoost21,AdaBoost22,Decision-Trees1,Decision-Trees11,Decision-Trees12,Decision-Trees21,...,xgboost11-0,xgboost11,xgboost12-0,xgboost12,xgboost21-0,xgboost21,xgboost22-0,xgboost22,sum,count
0,0013_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,False,False,False,31,38
1,0018_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,0,38
2,0019_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,37,38
3,0021_01,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,35,38
4,0023_01,True,False,False,True,True,False,True,True,True,...,True,True,True,True,True,True,True,True,34,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,True,30,38
4273,9269_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,2,38
4274,9271_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,37,38
4275,9273_01,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,True,33,38


In [114]:
dfe['Transported'] = 2 * dfe['sum'] >= dfe['count']

In [115]:
dfe

,PassengerId,AdaBoost1,AdaBoost11,AdaBoost12,AdaBoost21,AdaBoost22,Decision-Trees1,Decision-Trees11,Decision-Trees12,Decision-Trees21,...,xgboost11,xgboost12-0,xgboost12,xgboost21-0,xgboost21,xgboost22-0,xgboost22,sum,count,Transported
0,0013_01,True,True,True,True,True,True,True,True,True,...,True,True,True,False,False,False,False,31,38,True
1,0018_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,0,38,False
2,0019_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,37,38,True
3,0021_01,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,35,38,True
4,0023_01,True,False,False,True,True,False,True,True,True,...,True,True,True,True,True,True,True,34,38,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,True,True,True,True,True,False,False,True,...,True,True,True,True,True,True,True,30,38,True
4273,9269_01,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,2,38,False
4274,9271_01,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,37,38,True
4275,9273_01,True,True,True,True,True,True,True,True,False,...,True,True,True,True,True,True,True,33,38,True


In [116]:
dfe_f = dfe[['PassengerId', 'Transported']]

In [117]:
dfe_f.to_csv('..\\ensemble revb.csv', index=False)

In [ ]:
yhat = iso.fit_predict(X2)
mask = yhat != -1
outies = yhat == -1
# X_train, y_train = X_train_.loc[mask, :], y_train_.loc[mask, :]

In [ ]:
estimator.fit(X2.values, Y2.values)

In [ ]:
X_outies, Y_outies = X2.loc[outies, :], Y2.loc[outies, :]
X_inies, Y_inies = X2.loc[mask, :], Y2.loc[mask, :]

In [ ]:
len(X_outies)

In [ ]:
len(X_inies)

In [ ]:
xgb_model_ins = XGBClassifier(n_estimators=70)  # , num_parallel_tree=10)  #, n_jobs=100)
xgb_model_ins.fit(X_inies, Y_inies)
xgb_model_outs = XGBClassifier(n_estimators=70)  # , num_parallel_tree=10)  #, n_jobs=100)
xgb_model_outs.fit(X_outies, Y_outies)
# y_pred = xgb_model_ins.predict(X_test)
# predictions = [round(value) for value in y_pred]


In [ ]:
dft = pd.read_csv(files[1])  # the test data
dfct = transform_data_rev2(dft)
dfdt = dfct.loc[:, dfct.columns != 'PassengerId']  # passanger id should not be input for training
X2t = dfdt
print(len(X2t))

In [ ]:
yhat_t = iso.predict(X2t)
mask_t = yhat_t != -1
outies_t = yhat_t == -1

In [ ]:
x_test_ins = X2t.loc[mask_t, :]
x_test_outs = X2t.loc[outies_t, :]

In [ ]:
y_pred_ins = xgb_model_ins.predict(x_test_ins)
y_pred_outs = xgb_model_outs.predict(x_test_outs)

In [ ]:
dfcta = dfct[['PassengerId']]
pass_id_ins = dfcta.loc[mask_t, :]
pass_id_outs = dfcta.loc[outies_t, :]

pred_df_ins = pd.DataFrame(data=pass_id_ins, columns=['Transported'])
pred_df_ins['Transported'] = pred_df_ins['Transported'].astype(bool)
df_test_ins = pd.concat([pass_id_ins, pred_df_ins], axis=1)

pred_df_outs = pd.DataFrame(data=pass_id_outs, columns=['Transported'])
pred_df_outs['Transported'] = pred_df_outs['Transported'].astype(bool)
df_test_outs = pd.concat([pass_id_outs, pred_df_outs], axis=1)

In [ ]:
df_test_ins.to_csv('..\\revA\\xgboost_ins.csv', index=False)
df_test_outs.to_csv('..\\revA\\xgboost_outs.csv', index=False)

# Testing iso

In [ ]:
dfc2

In [ ]:
iso2 = IsolationForest(contamination=0.1)
yhat = iso2.fit_predict(dfc2)
mask = yhat != -1
outies = yhat == -1

In [ ]:
outliers_train = dfc2[outies]
outliers_train['Transported'] = outliers_train['Transported'].astype(int)

In [ ]:
outliers_train.to_csv('..\\outliers_iso_train.csv', index=False)

In [ ]:
dtale.show(outliers_train)

In [ ]:
ddff = dfc2
ddff['Transported'] = ddff['Transported'].astype(int)

In [ ]:
dtale.show(ddff)